In [1]:
import os
from multiprocessing import Pool, Manager
import pandas as pd
from geopy import distance
from datetime import timedelta
import planetary_computer as pc
from pystac_client import Client
import rioxarray
import numpy as np
import cv2
from tqdm import tqdm
import warnings

In [2]:
catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1", modifier=pc.sign_inplace
)

In [3]:
# get our bounding box to search latitude and longitude coordinates
def get_bounding_box(latitude, longitude, meter_buffer=3000):
    """
    Given a latitude, longitude, and buffer in meters, returns a bounding
    box around the point with the buffer on the left, right, top, and bottom.

    Returns a list of [minx, miny, maxx, maxy]
    """
    distance_search = distance.distance(meters=meter_buffer)

    # calculate the lat/long bounds based on ground distance
    # bearings are cardinal directions to move (south, west, north, and east)
    min_lat = distance_search.destination((latitude, longitude), bearing=180)[0]
    min_long = distance_search.destination((latitude, longitude), bearing=270)[1]
    max_lat = distance_search.destination((latitude, longitude), bearing=0)[0]
    max_long = distance_search.destination((latitude, longitude), bearing=90)[1]

    return [min_long, min_lat, max_long, max_lat]

In [4]:
# get our date range to search, and format correctly for query
def get_date_range(date, time_buffer_days=15):
    """Get a date range to search for in the planetary computer based
    on a sample's date. The time range will include the sample date
    and time_buffer_days days prior

    Returns a string"""
    datetime_format = "%Y-%m-%d"
    range_start = pd.to_datetime(date) - timedelta(days=time_buffer_days)
    date_range = f"{range_start.strftime(datetime_format)}/{pd.to_datetime(date).strftime(datetime_format)}"

    return date_range

In [5]:
def crop_sentinel_image(item, bounding_box, band):
    """
    Given a STAC item from Sentinel-2 and a bounding box tuple in the format
    (minx, miny, maxx, maxy), return a cropped portion of the item's visual
    imagery in the bounding box.

    Returns the image as a numpy array with dimensions (color band, height, width)
    """
    (minx, miny, maxx, maxy) = bounding_box

    image = rioxarray.open_rasterio(pc.sign(item.assets[band].href)).rio.clip_box(
        minx=minx,
        miny=miny,
        maxx=maxx,
        maxy=maxy,
        crs="EPSG:4326",
    )

    return image.to_numpy()

In [6]:
# Get images
def get_images(row):
    """
    Given a row from the metadada, return 2 cropped images from sentinel
    1. True color image
    2. Water mask
    """
    bbox = get_bounding_box(row.latitude, row.longitude, meter_buffer=3000)
    date_range = get_date_range(row.date)
    
    # search the planetary computer sentinel-l2a and landsat level-2 collections
    search = catalog.search(
        collections=["sentinel-2-l2a"], 
        bbox=bbox, 
        datetime=date_range,
        query={"eo:cloud_cover": {"lt": 10}}
    )
    
    # get items
    items = [item for item in search.item_collection()]
    
    # get details of all of the items returned
    item_details = pd.DataFrame(
        [
            {
                "datetime": item.datetime.strftime("%Y-%m-%d"),
                "platform": item.properties["platform"],
                "min_long": item.bbox[0],
                "max_long": item.bbox[2],
                "min_lat": item.bbox[1],
                "max_lat": item.bbox[3],
                "bbox": item.bbox,
                "item_obj": item,
            }
            for item in items
        ]
    )
    
    # check which rows actually contain the sample location
    item_details["contains_sample_point"] = (
        (item_details.min_lat < row.latitude)
        & (item_details.max_lat > row.latitude)
        & (item_details.min_long < row.longitude)
        & (item_details.max_long > row.longitude)
    )
    item_details = item_details[item_details["contains_sample_point"]]
    item_details[["datetime", "platform", "contains_sample_point", "bbox"]].sort_values(
        by="datetime"
    )
    
    #Get best item
    best_item = (
    item_details[item_details.platform.str.contains("Sentinel")]
    .sort_values(by="datetime", ascending=False)
    .iloc[0]
    )
    item = best_item.item_obj

    bbox = get_bounding_box(row.latitude, row.longitude, meter_buffer=1000)
    true_color = crop_sentinel_image(item, bbox, "visual")
    scl = crop_sentinel_image(item, bbox, "SCL")[0]
    
    # transpose
    visual = np.transpose(true_color, axes=[1, 2, 0])
    water_mask = np.stack([cv2.resize(scl, (visual.shape[1], visual.shape[0]))] * 3, -1) == 6
    water_mask = np.where(water_mask, 1, np.nan)
    water_mask = np.mean(water_mask, axis=2, keepdims=True)
    
    # Return images
    return visual, water_mask

In [7]:
def get_features(row):
    visual, water_mask = get_images(row)
    visual = visual * water_mask
    red_values = visual[:,:,0].flatten()
    green_values = visual[:,:,1].flatten()
    blue_values = visual[:,:,2].flatten() 
    # Calcular los histogramas de cada banda de color
    red_hist, red_bins = np.histogram(red_values, bins=255, range=(0, 255))
    green_hist, green_bins = np.histogram(green_values, bins=255, range=(0, 255))
    blue_hist, blue_bins = np.histogram(blue_values, bins=255, range=(0, 255))
    #
    means = {
        'uid': row.uid,
        'R_mean': np.nanmean(red_values),
        'G_mean': np.nanmean(green_values),
        'B_mean': np.nanmean(blue_values),
        'R_median': np.nanmedian(red_values),
        'G_median': np.nanmedian(green_values),
        'B_median': np.nanmedian(blue_values),
        'G_vs_R': np.nanmean(green_values) / np.nanmean(red_values),
        'G_vs_B': np.nanmean(green_values) / np.nanmean(blue_values),
    }
    blue_f = {'B_' + str(int(x)): 0 if np.isnan(y) else y for x, y in zip(blue_bins, blue_values)}
    green_f = {'G_' + str(int(x)): 0 if np.isnan(y) else y for x, y in zip(green_bins, green_values)}
    red_f = {'R_' + str(int(x)): 0 if np.isnan(y) else y for x, y in zip(red_bins, red_values)}
    
    return dict(means, **blue_f, **green_f, **red_f)

In [8]:
# Save Image
def save_features(row, failed_points, features):
    """
    Given a row, save the features generated
    """
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("error")
            features.append(get_features(row))
    except Exception as e:
        failed_points.append({'uid': row['uid']})

In [9]:
metadata = pd.read_csv('../data/metadata.csv')

In [10]:
%%time
def save_features_wrapper(args):
    r, valid_points, features = args
    save_features(r, failed_points, features)

# Utilizamos una lista compartida para almacenar los puntos válidos
manager = Manager()
failed_points = manager.list()
features = manager.list()

head = len(metadata)

# Obtener el número total de filas
total_rows = len(metadata.head(head))

# Crear un iterable de argumentos para el método map
args = [(r, failed_points, features) for _, r in metadata.head(head).iterrows()]

# Crear un Pool de procesos
with Pool(processes=32) as pool:
    # Utilizar tqdm para la barra de progreso
    with tqdm(total=total_rows) as pbar:
        # Mapear la función sobre los argumentos
        for _ in pool.imap_unordered(save_features_wrapper, args):
            pbar.update(1)


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 23570/23570 [39:58<00:00,  9.83it/s]

CPU times: user 13.8 s, sys: 2.99 s, total: 16.8 s
Wall time: 39min 58s


In [11]:
features_df = pd.DataFrame.from_records(features)
features_df.to_csv('../data/downloaded/sentinel/features.csv')

In [12]:
failed_df = pd.DataFrame.from_records(failed_points)
failed_df.to_csv('../data/downloaded/failed/sentinel.csv')

In [13]:
len(failed_points)

14094

In [14]:
len(features)

9476

In [15]:
len(failed_points)/len(metadata.head(head))

0.5979635129401782

In [16]:
len(features)/len(metadata.head(head))

0.4020364870598218